In [ ]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
import tensorflow as tf
import random

import warnings
warnings.filterwarnings("ignore", category=Warning)

In [ ]:
# Connect to Google Drive
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

In [ ]:
# Change directory to ndvi folder
%cd drive/MyDrive/ndvi

In [ ]:
# Add the data files in the folder into a list and print it out
ndvi_files_list = []
for file in os.listdir():
  if file.endswith('.csv'):
    ndvi_files_list.append(file)

print("List of Files in NDVI Folder:")
print(*ndvi_files_list, sep = "\n")

In [ ]:
# Read the combined ndvi data file
ndvi_df = pd.read_csv("NDVI_Complete.csv")
display(ndvi_df)

In [ ]:
# Get current directory
current_directory = os.getcwd()
print("Current Directory:", current_directory)

In [ ]:
# Change directory to parent directory
%cd ..

In [ ]:
# Change directory to gnvi folder
%cd gndvi

In [ ]:
# Add the data files in the folder into a list and print it out
gndvi_files_list = []
for file in os.listdir():
  if file.endswith('.csv'):
    gndvi_files_list.append(file)

print("List of Files in GNDVI Folder:")
print(*gndvi_files_list, sep = "\n")

In [ ]:
# Read the combined gndvi data file
gndvi_df = pd.read_csv("GNDVI_Complete.csv")
display(gndvi_df)

In [ ]:
# Get current directory
current_directory = os.getcwd()
print("Current Directory:", current_directory)

In [ ]:
# Change directory to parent directory
%cd ..

In [ ]:
# Change directory to no2 folder
%cd no2

In [ ]:
# Add the data files in the folder into a list and print it out
no2_files_list = []
for file in os.listdir():
  if file.endswith('.csv'):
    no2_files_list.append(file)

print("List of Files in NO2 Folder:")
print(*no2_files_list, sep = "\n")

In [ ]:
# Read the combined no2 data file
no2_df = pd.read_csv("NO2_Complete.csv")
display(no2_df)

In [ ]:
# Get current directory
current_directory = os.getcwd()
print("Current Directory:", current_directory)

In [ ]:
# Change directory to parent directory
%cd ..

In [ ]:
# Change directory to soil moisture folder
%cd soil_moisture

In [ ]:
# Add the data files into a list and print it out
soil_moisture_files_list = []
for file in os.listdir():
  if file.endswith('.csv'):
    soil_moisture_files_list.append(file)

print("List of Files in Soil Moisture Folder:")
print(*soil_moisture_files_list, sep = "\n")

In [ ]:
# Read the combined soil moisture data file
soil_moisture_df = pd.read_csv("Soil_Moisture_Complete.csv")
display(soil_moisture_df)

In [ ]:
# Get current directory
current_directory = os.getcwd()
print("Current Directory:", current_directory)

In [ ]:
# Change directory to parent directory
%cd ..

In [ ]:
# Change directory to soil temperature folder
%cd soil_temp

In [ ]:
# Add the data files in the folder into a list and print it out
soil_temp_files_list = []
for file in os.listdir():
  if file.endswith('.csv'):
    soil_temp_files_list.append(file)

print("List of Files in Soil Temperature Folder:")
print(*soil_temp_files_list, sep = "\n")

In [ ]:
# Read the combined soil temperature data file
soil_temp_df = pd.read_csv("Soil_Temp_Complete.csv")
display(soil_temp_df)

In [ ]:
# Remove the date column in gndvi dataframe
gndvi_df = gndvi_df.drop(columns=['date'])

# Merge ndvi and gdnvi dataframes together
ndvi_gndvi_df = pd.concat([ndvi_df, gndvi_df], axis=1)

# Display the dataframe after merging
display(ndvi_gndvi_df)

In [ ]:
# Remove any rows where there is missing values in 'no2' column
no2_filled_df = no2_df.copy()

# Perform the interpolation on the copied dataframe
no2_filled_df['no2'] = no2_filled_df['no2'].interpolate(method="linear")

# Print number of duplicated rows based the 'date' column
no2_duplicatedRows_sum = no2_filled_df.duplicated(['date']).sum()
print("Number of duplicated rows in NO2 dataframe:", no2_duplicatedRows_sum)

In [ ]:
# Remove duplicated rows based the 'date' column
no2_filled_df = no2_filled_df.drop_duplicates(subset=['date'], keep='last')

# Print number of duplicated rows based the 'date' column after duplicates are removed
no2_duplicatedRows_sum = no2_filled_df.duplicated(['date']).sum()
print("Number of duplicated rows in NO2 dataframe:", no2_duplicatedRows_sum)

In [ ]:
# Convert 'date' column to datetime object
ndvi_gndvi_df['date'] = pd.to_datetime(ndvi_gndvi_df['date'])
no2_filled_df['date'] = pd.to_datetime(no2_filled_df['date'])

# Merge the ndvi_gndvi with no2 dataframes based on the 'date' column
ndvi_gndvi_no2_df = pd.merge(ndvi_gndvi_df, no2_filled_df, on='date', how='inner')

# Display the merged dataframe
display(ndvi_gndvi_no2_df)

In [ ]:
# Remove any rows where there is missing values in 'soil_moisture' column
soil_moisture_df = soil_moisture_df.dropna(subset=['soil_moisture'])

# Print number of duplicated rows based the 'date' column
soil_moisture_duplicatedRows_sum = soil_moisture_df.duplicated(['date']).sum()
print("Number of duplicated rows in soil moisture dataframe:", soil_moisture_duplicatedRows_sum)

In [ ]:
# Convert 'date' column to datetime object
soil_moisture_df['date'] = pd.to_datetime(soil_moisture_df['date'])

# Merge ndvi_gndvi_no2 with soil_moisture dataframes based on the 'date' column
ndvi_gndvi_no2_soilMoisture_df = pd.merge(ndvi_gndvi_no2_df, soil_moisture_df, on='date', how='inner')

# Display the merged dataframe
display(ndvi_gndvi_no2_soilMoisture_df)

In [ ]:
# Remove any rows where there is missing values in 'soil_temp' column
soil_temp_df = soil_temp_df.dropna(subset=['soil_temp'])

# Print number of duplicated rows based the 'date' column
soil_temp_duplicatedRows_sum = soil_temp_df.duplicated(['date']).sum()
print("Number of duplicated rows in soil temperature dataframe:", soil_temp_duplicatedRows_sum)

In [ ]:
# Convert 'date' column to datetime object
soil_temp_df['date'] = pd.to_datetime(soil_temp_df['date'])

# Merge ndvi_gndvi_no2_soilMoisture with soil_temp dataframes based on the 'date' column
merged_df = pd.merge(ndvi_gndvi_no2_soilMoisture_df, soil_temp_df, on='date', how='inner')

# Display the merged dataframe
display(merged_df)

merged_df.to_csv("merged.csv")

In [ ]:
# Remove duplicated rows based on the 'date' column
merged_dropped_df = merged_df.drop_duplicates('date', keep='last')
display(merged_dropped_df)

In [ ]:
# Check for any null values
merged_dropped_df.isnull().sum()

In [ ]:
# Set 'date' column as the index
data_df = merged_dropped_df.set_index('date')

# Create new columns of 'day', 'month', and 'year'
data_df['day'] = data_df.index.day
data_df['month'] = data_df.index.month
data_df['year'] = data_df.index.year
display(data_df)

In [ ]:
# Create new column 'growing_season' based on the time period
data_df['growing_season'] = np.where(
    (data_df['month'] >= 3) & (data_df['month'] <= 5),
    1,  # March to May
    np.where(
        (data_df['month'] >= 9) & (data_df['month'] <= 11),
        2,  # September to November
        np.nan
    )
)

data_df['growing_season'] = data_df['growing_season'].astype('Int64')

# Display the dataframe
display(data_df)

In [ ]:
# Function to add yield value to each group
def add_yield_value(group):
    if group['year'].iloc[0] == 2019 and group['growing_season'].iloc[0] == 1:
        yield_value = 20686.4
    elif group['year'].iloc[0] == 2019 and group['growing_season'].iloc[0] == 2:
        yield_value = 19324.8
    elif group['year'].iloc[0] == 2020 and group['growing_season'].iloc[0] == 1:
        yield_value = 19985.7
    elif group['year'].iloc[0] == 2020 and group['growing_season'].iloc[0] == 2:
        yield_value = 21245.3
    elif group['year'].iloc[0] == 2021 and group['growing_season'].iloc[0] == 1:
        yield_value = 17222.4
    elif group['year'].iloc[0] == 2021 and group['growing_season'].iloc[0] == 2:
        yield_value = 19249.4

    # Initialise the 'yield' column with value 0
    group['yield'] = 0

    # Assign the yield value to the last row of its corresponding group
    group['yield'].iloc[-1] = yield_value

    return group

# Apply function to add the yield value to the dataframe
complete_data_df = data_df.groupby(['year', 'growing_season'], group_keys=False).apply(add_yield_value)

# Display the dataframe
display(complete_data_df)

In [ ]:
# Get current directory
current_directory = os.getcwd()
print("Current Directory:", current_directory)

In [ ]:
# Change directory to parent directory
%cd ..

In [ ]:
# Export dataframe to csv in Google Drive
complete_data_df.to_csv("Data.csv")

In [ ]:
# Read Data csv file as dataframe
data = pd.read_csv("Data.csv")
display(data)

In [ ]:
# Obtain the minimum number of rows out of all the groups
min_row_count = data.groupby(['year', 'growing_season']).size().min()
print("Period:", min_row_count)

In [ ]:
# Remove additional rows to ensure each group has same number of rows by keeping the tail of each group
data = data.groupby(['year', 'growing_season']).tail(min_row_count)
display(data)

In [ ]:
# Function to display the observed values for each feature
def decompose_observed(df, features, period):
    # Create subplots for each feature
    fig, axis = plt.subplots(len(features), figsize=(16,10))

    # Adjust the vertical space between each subplots
    plt.subplots_adjust(hspace=0.5)

    # Add title
    fig.suptitle('Observed Values', fontsize=20)

    # Loop to plot the observed value for each feature
    for i, feature in enumerate(features):
        # Decomposing time series value into time series components
        decomposed = seasonal_decompose(df[feature].values, period=period)

        # Get the component of observed
        observed = decomposed.observed

        # Plot the observed values for each feature
        axis[i].set_title(f'Observed ({feature})', fontsize=16)
        axis[i].plot(observed)
        axis[i].grid()

    # Display the plot
    plt.show()

# Function to display the trend of each feature
def decompose_trend(df, features, period):
    # Create subplots for each feature
    fig, axis = plt.subplots(len(features), figsize=(16,10))

    # Adjust the vertical space between subplots
    plt.subplots_adjust(hspace=0.5)

    # Add title
    fig.suptitle('Trend', fontsize=20)

    # Loop to plot the trend for each feature
    for i, feature in enumerate(features):
        # Decomposing time series value into time series components
        decomposed = seasonal_decompose(df[feature].values, period=period)

        # Get the component of trend
        trend = decomposed.trend

        # Plot the trend of each feature
        axis[i].set_title(f'Trend ({feature})', fontsize=16)
        axis[i].plot(trend)
        axis[i].grid()

    # Display the plot
    plt.show()

# Function to display the seasonality of each feature
def decompose_seasonality(df, features, period):
    # Create subplots for each feature
    fig, axis = plt.subplots(len(features), figsize=(16,10))

    # Adjust the vertical space between subplots
    plt.subplots_adjust(hspace=0.5)

    # Add title
    fig.suptitle('Seasonality', fontsize=20)

    # Loop to plot the seasonality for each feature
    for i, feature in enumerate(features):
        # Decomposing time series value into time series components
        decomposed = seasonal_decompose(df[feature].values, period=period)

        # Get the component of seasonality
        seasonality = decomposed.seasonal

        # Plot the seasonality of each feature
        axis[i].set_title(f'Seasonality ({feature})', fontsize=16)
        axis[i].plot(seasonality)
        axis[i].grid()

    # Display the plot
    plt.show()

# Function to display the residual of each feature
def decompose_residual(df, features, period):
    # Create subplots for each feature
    fig, axis = plt.subplots(len(features), figsize=(16,10))

    # Adjust the vertical space between subplots
    plt.subplots_adjust(hspace=0.5)

    # Add title
    fig.suptitle('Residual', fontsize=20)

    # Loop to plot the residual for each feature
    for i, feature in enumerate(features):
        # Decomposing time series value into time series components
        decomposed = seasonal_decompose(df[feature].values, period=period)

        # Get the component of residual
        residual = decomposed.resid

        # Plot the residual of each feature
        axis[i].set_title(f'Residual ({feature})', fontsize=16)
        axis[i].plot(residual)
        axis[i].grid()

    # Display the plot
    plt.show()

In [ ]:
# Call decompose_observed function
decompose_observed(data, ['ndvi', 'gndvi', 'no2', 'soil_moisture', 'soil_temp'], 14)

In [ ]:
# Call decompose_trend function
decompose_trend(data, ['ndvi', 'gndvi', 'no2', 'soil_moisture', 'soil_temp'], 14)

In [ ]:
# Call decompose_seasonality function
decompose_seasonality(data, ['ndvi', 'gndvi', 'no2', 'soil_moisture', 'soil_temp'], 14)

In [ ]:
# Call decompose_residual function
decompose_residual(data, ['ndvi', 'gndvi', 'no2', 'soil_moisture', 'soil_temp'], 14)

In [ ]:
# Function to identify and treat the outliers
def identify_and_treatment_outliers(df, features, period):
    # Decomposing time series value into time series components
    res = seasonal_decompose(df[features], period=period)

    # Get the component of residual
    residual = res.resid

    # Calculate the first quartile, second quartile, and IQR
    q1 = residual.quantile(0.25)
    q3 = residual.quantile(0.75)
    iqr = q3 - q1

    # Identify the outliers
    outliers = (residual > (q3 + (iqr * 1.5))) | (residual < (q1 - (iqr * 1.5)))

    # Replace outliers values with linear interpolated values
    residual[outliers] = residual.interpolate(method="linear")

In [ ]:
# Treat outliers by calling the identify_and_treatment_outliers function
identify_and_treatment_outliers(data, 'ndvi', 14)
identify_and_treatment_outliers(data, 'gndvi', 14)
identify_and_treatment_outliers(data, 'no2', 14)
identify_and_treatment_outliers(data, 'soil_moisture', 14)
identify_and_treatment_outliers(data, 'soil_temp', 14)

In [ ]:
# Features for which normalisations will be performed on
features_normalise = ['ndvi', 'gndvi', 'no2', 'soil_moisture', 'soil_temp', 'yield']

# Normalize the features using MinMaxScaler
scaler = MinMaxScaler()
data[features_normalise] = scaler.fit_transform(data[features_normalise])

# Display the dataframe after normalisation
display(data)

In [ ]:
# Check the current directory
current_directory = os.getcwd()
print("Current Directory:", current_directory)

In [ ]:
# Export the dataframe into csv file
data.to_csv("Data_Training.csv")

In [ ]:
# Random Forest Regression

# Read the data csv file into a dataframe
rf_data_df = pd.read_csv("Data_Training.csv")

# Features where lagging will be done
rf_features_to_lag = ['ndvi', 'gndvi', 'no2', 'soil_moisture', 'soil_temp']

# Specify the lag days
rf_lag_days = 13

# Create lagged columns within each group defined by 'year' and 'growing_season'
def lag_days_range(x):
    return range(1, len(x) + 1)

# Create lagged columns for each feature in 'features_to_lag'
for feature in rf_features_to_lag:
    # Create '{column}_lag_days' column
    rf_data_df[f'{feature}_lag_days'] = rf_data_df.groupby(['year', 'growing_season'])[feature].transform(lag_days_range)

    # Create lagged columns for each day
    for i in range(1, rf_lag_days + 1):
        rf_data_df[f'{feature}_lagged_{i}'] = rf_data_df.groupby(['year', 'growing_season'])[feature].shift(i)

# Convert lagged columns to float
rf_lagged_columns = [f'{feature}_lagged_{lag}' for feature in rf_features_to_lag for lag in range(1, rf_lag_days + 1)]
rf_data_df[rf_lagged_columns] = rf_data_df[rf_lagged_columns].astype(float)

# Drop rows with missing values
rf_data_df = rf_data_df.dropna()

# Split the data into training and testing sets based on the year
rf_unique_year_count = len(rf_data_df['year'].unique())
rf_train_years = int(rf_unique_year_count * 0.8)
rf_split_index = rf_data_df['year'].unique()[rf_train_years - 1]
rf_train = rf_data_df[rf_data_df['year'] <= rf_split_index].dropna(subset=['yield'])
rf_test = rf_data_df[rf_data_df['year'] > rf_split_index].dropna(subset=['yield'])

# Define the features and target variable
rf_features = rf_lagged_columns + ['ndvi', 'gndvi', 'no2', 'soil_moisture', 'soil_temp', 'day','month','year','growing_season']
rf_target = 'yield'

# Drop rows with missing values
rf_data_df = rf_data_df.dropna(subset=rf_features)

# Define the train and test data
rf_X_train, rf_y_train = rf_train[rf_features], rf_train[rf_target]
rf_X_test, rf_y_test = rf_test[rf_features], rf_test[rf_target]

# Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(rf_X_train, rf_y_train)

# Make predictions on the testing set
rf_y_pred = rf_model.predict(rf_X_test)

# Evaluate the RF model performance
rf_mse = mean_squared_error(rf_y_test, rf_y_pred)
rf_mse_perHectare = (rf_mse/4435.343)*1000

rf_rmse = np.sqrt(mean_squared_error(rf_y_test, rf_y_pred))
rf_rmse_perHectare = (rf_rmse/4435.343)*1000

rf_mae = mean_absolute_error(rf_y_test, rf_y_pred)
rf_mae_perHectare = (rf_mae/4435.343)*1000

print('Model Performance of Random Forest\n')
print(f'Mean Squared Error (MSE): {rf_mse_perHectare:.6f} kg/ha')
print(f'Root Mean Squared Error (RMSE): {rf_rmse_perHectare:.6f} kg/ha')
print(f'Mean Absolute Error (MAE): {rf_mae_perHectare:.6f} kg/ha')

In [ ]:
# LSTM for Growing Season 1

# Set random seeds
seed_value = 42
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
random.seed(seed_value)

# Read the data csv file into a dataframe
lstm_gs1_data_df = pd.read_csv("Data_Training.csv")

# Change 'year' to a categorical variable
lstm_gs1_data_df['year'] = lstm_gs1_data_df['year'].astype('category')

# Define the number of time steps
time_steps_lstm_gs1 = 14
unique_years_lstm_gs1 = lstm_gs1_data_df['year'].unique()

# Split the data into training and testing sets based on the year
split_index_lstm_gs1 = int(0.8 * len(unique_years_lstm_gs1))
train_years_lstm_gs1, test_years_lstm_gs1 = unique_years_lstm_gs1[:split_index_lstm_gs1], unique_years_lstm_gs1[split_index_lstm_gs1:]

# Store the training data
X_train_list_lstm_gs1 = []
y_train_list_lstm_gs1 = []

# Loop through training years to accumulate training data
for year in train_years_lstm_gs1:
    # Get the data for a group based on growing season and year
    group_df = lstm_gs1_data_df[(lstm_gs1_data_df['growing_season'] == 1) & (lstm_gs1_data_df['year'] == year)].copy()

    # Define the features and target variable
    target_lstm_gs1 = 'yield'
    features_lstm_gs1 = ['ndvi', 'gndvi', 'no2', 'soil_moisture', 'soil_temp']

    # Create a new column 'year_feature'
    group_df['year_feature'] = group_df['year'].cat.codes

    # Get the features values and target variable value
    X_lstm_gs1 = group_df[features_lstm_gs1 + ['year_feature']].values
    y_lstm_gs1 = group_df[target_lstm_gs1].values[-1]

    X_train_list_lstm_gs1.append(X_lstm_gs1)
    y_train_list_lstm_gs1.append(y_lstm_gs1)

# Combine the training data across the years
X_train_lstm_gs1 = np.concatenate(X_train_list_lstm_gs1)
y_train_lstm_gs1 = np.array(y_train_list_lstm_gs1)

# Reshape the input data for LSTM
X_train_lstm_gs1 = X_train_lstm_gs1.reshape((X_train_lstm_gs1.shape[0] // time_steps_lstm_gs1, time_steps_lstm_gs1, X_train_lstm_gs1.shape[1]))

# Build LSTM Model
model_lstm_gs1 = Sequential()
model_lstm_gs1.add(LSTM(units=50, input_shape=(time_steps_lstm_gs1, X_train_lstm_gs1.shape[2])))
model_lstm_gs1.add(Dense(units=1))

# Compile the model
model_lstm_gs1.compile(optimizer='adam', loss='mean_squared_error')

# Fit the model with the training data
model_lstm_gs1.fit(X_train_lstm_gs1, y_train_lstm_gs1, epochs=100, batch_size=32)

# Test the model on testing data
for year in test_years_lstm_gs1:
    # Get the data for a group based on growing season and year
    group_df = lstm_gs1_data_df[(lstm_gs1_data_df['growing_season'] == 1) & (lstm_gs1_data_df['year'] == year)].copy()

    # Define the features and target variable
    target_lstm_gs1 = 'yield'
    features_lstm_gs1 = ['ndvi', 'gndvi', 'no2', 'soil_moisture', 'soil_temp']

    # Create a new column 'year_feature'
    group_df['year_feature'] = group_df['year'].cat.codes

    # Get the features values and target variable value
    X_test_lstm_gs1 = group_df[features_lstm_gs1 + ['year_feature']].values
    y_test_lstm_gs1 = group_df[target_lstm_gs1].values[-1]

    # Reshape the input data for LSTM
    X_test_lstm_gs1 = X_test_lstm_gs1.reshape((1, time_steps_lstm_gs1, X_test_lstm_gs1.shape[1]))

    # Get the predictions of the model
    predictions_lstm_gs1 = model_lstm_gs1.predict(X_test_lstm_gs1)

    # Evaluate the LSTM model
    lstm_gs1_mse = mean_squared_error(np.array([y_test_lstm_gs1]), predictions_lstm_gs1)
    lstm_gs1_mse_perHectare = (lstm_gs1_mse/4435.343)*1000

    lstm_gs1_rmse = np.sqrt(lstm_gs1_mse)
    lstm_gs1_rmse_perHectare = (lstm_gs1_rmse/4435.343)*1000

    lstm_gs1_mae = mean_absolute_error(np.array([y_test_lstm_gs1]), predictions_lstm_gs1)
    lstm_gs1_mae_perHectare = (lstm_gs1_mae/4435.343)*1000

    print('\nModel Performance of LSTM Growing Season 1\n')
    print(f'Mean Squared Error (MSE): {lstm_gs1_mse_perHectare:.6f} kg/ha')
    print(f'Root Mean Squared Error (RMSE): {lstm_gs1_rmse_perHectare:.6f} kg/ha')
    print(f'Mean Absolute Error (MAE): {lstm_gs1_mae_perHectare:.6f} kg/ha')

In [ ]:
# LSTM for Growing Season 2

# Set random seeds
seed_value = 42
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
random.seed(seed_value)

# Read the data csv file into a dataframe
lstm_gs2_data_df = pd.read_csv("Data_Training.csv")

# Change 'year' to a categorical variable
lstm_gs2_data_df['year'] = lstm_gs2_data_df['year'].astype('category')

# Define the number of time steps
time_steps_lstm_gs2 = 14
unique_years_lstm_gs2 = lstm_gs2_data_df['year'].unique()

# Split the data into training and testing sets based on the year
split_index_lstm_gs2 = int(0.8 * len(unique_years_lstm_gs2))
train_years_lstm_gs2, test_years_lstm_gs2 = unique_years_lstm_gs2[:split_index_lstm_gs2], unique_years_lstm_gs2[split_index_lstm_gs2:]

# Store the training data
X_train_list_lstm_gs2 = []
y_train_list_lstm_gs2 = []

# Loop through training years to accumulate training data
for year in train_years_lstm_gs2:
    # Get the data for a group based on growing season and year
    group_df = lstm_gs2_data_df[(lstm_gs2_data_df['growing_season'] == 2) & (lstm_gs2_data_df['year'] == year)].copy()

    # Define the features and target variable
    target_lstm_gs2 = 'yield'
    features_lstm_gs2 = ['ndvi', 'gndvi', 'no2', 'soil_moisture', 'soil_temp']

    # Create a new column 'year_feature'
    group_df['year_feature'] = group_df['year'].cat.codes

    # Get the features values and target variable value
    X_lstm_gs2 = group_df[features_lstm_gs2 + ['year_feature']].values
    y_lstm_gs2 = group_df[target_lstm_gs2].values[-1]

    X_train_list_lstm_gs2.append(X_lstm_gs2)
    y_train_list_lstm_gs2.append(y_lstm_gs2)

# Combine the training data across the years
X_train_lstm_gs2 = np.concatenate(X_train_list_lstm_gs2)
y_train_lstm_gs2 = np.array(y_train_list_lstm_gs2)

# Reshape the input data for LSTM
X_train_lstm_gs2 = X_train_lstm_gs2.reshape((X_train_lstm_gs2.shape[0] // time_steps_lstm_gs2, time_steps_lstm_gs2, X_train_lstm_gs2.shape[1]))

# Build LSTM Model
model_lstm_gs2 = Sequential()
model_lstm_gs2.add(LSTM(units=50, input_shape=(time_steps_lstm_gs2, X_train_lstm_gs2.shape[2])))
model_lstm_gs2.add(Dense(units=1))

# Compile the model
model_lstm_gs2.compile(optimizer='adam', loss='mean_squared_error')

# Fit the model with the training data
model_lstm_gs2.fit(X_train_lstm_gs2, y_train_lstm_gs2, epochs=100, batch_size=32)

# Test the model on testing data
for year in test_years_lstm_gs2:
    # Get the data for a group based on growing season and year
    group_df = lstm_gs2_data_df[(lstm_gs2_data_df['growing_season'] == 2) & (lstm_gs2_data_df['year'] == year)].copy()

    # Define the features and target variable
    target_lstm_gs2 = 'yield'
    features_lstm_gs2 = ['ndvi', 'gndvi', 'no2', 'soil_moisture', 'soil_temp']

    # Create a new column 'year_feature'
    group_df['year_feature'] = group_df['year'].cat.codes

    # Get the features values and target variable value
    X_test_lstm_gs2 = group_df[features_lstm_gs2 + ['year_feature']].values
    y_test_lstm_gs2 = group_df[target_lstm_gs2].values[-1]

    # Reshape the input data for LSTM
    X_test_lstm_gs2 = X_test_lstm_gs2.reshape((1, time_steps_lstm_gs2, X_test_lstm_gs2.shape[1]))

    # Get the predictions of the model
    predictions_lstm_gs2 = model_lstm_gs2.predict(X_test_lstm_gs2)

    # Evaluate the LSTM model
    lstm_gs2_mse = mean_squared_error(np.array([y_test_lstm_gs2]), predictions_lstm_gs2)
    lstm_gs2_mse_perHectare = (lstm_gs2_mse/4435.343)*1000

    lstm_gs2_rmse = np.sqrt(lstm_gs2_mse)
    lstm_gs2_rmse_perHectare = (lstm_gs2_rmse/4435.343)*1000

    lstm_gs2_mae = mean_absolute_error(np.array([y_test_lstm_gs2]), predictions_lstm_gs2)
    lstm_gs2_mae_perHectare = (lstm_gs2_mae/4435.343)*1000

    print('\nModel Performance of LSTM Growing Season 2\n')
    print(f'Mean Squared Error (MSE): {lstm_gs2_mse_perHectare:.6f} kg/ha')
    print(f'Root Mean Squared Error (RMSE): {lstm_gs2_rmse_perHectare:.6f} kg/ha')
    print(f'Mean Absolute Error (MAE): {lstm_gs2_mae_perHectare:.6f} kg/ha')